In [5]:
import pandas as pd

#datetime to parse dates
from datetime import datetime
from datetime import date

# Define the file path (adjust the date string to match your file)
file_path = r"C:\Users\Kaity\Desktop\SchoolActivityDays\SANrawdata_2025-08-25.csv"

# Load the CSV into a DataFrame
sport_calendar_df = pd.read_csv(file_path)

# Quick checks
print(sport_calendar_df.shape)
print(sport_calendar_df.head())


(5026, 4)
          Sport        Date    Time                 School
0          Golf  2024-08-05     TBD  Riverbend High School
1          Golf  2024-08-07     TBD  Riverbend High School
2          Golf  2024-08-14  4:00pm  Riverbend High School
3  Field Hockey  2024-08-14  5:00pm  Riverbend High School
4  Field Hockey  2024-08-14  6:30pm  Riverbend High School


In [29]:
import os

# Set working directory to your folder
os.chdir(r"C:\Users\Kaity\Desktop\SchoolActivityDays")

# Confirm it's set correctly
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\Kaity\Desktop\SchoolActivityDays


In [7]:
# Make sure the 'date' column is in datetime format
sport_calendar_df['Date'] = pd.to_datetime(sport_calendar_df['Date'])

# Create a new column with the day of the week (e.g., Monday, Tuesday, etc.)
sport_calendar_df['Weekday'] = sport_calendar_df['Date'].dt.day_name()

# Preview
print(sport_calendar_df[['Date', 'Weekday']].head())

        Date    Weekday
0 2024-08-05     Monday
1 2024-08-07  Wednesday
2 2024-08-14  Wednesday
3 2024-08-14  Wednesday
4 2024-08-14  Wednesday


In [9]:
sport_calendar_df['Sport'].unique()

array(['Golf', 'Field Hockey', 'Football', 'Volleyball',
       'Sideline Cheer Fall', 'Cross Country', 'Competition Cheer',
       'Scholastic Bowl', 'Basketball', 'Wrestling', 'Swim & Dive',
       'Indoor Track', 'Speech', 'Lacrosse', 'Tennis', 'Softball',
       'Baseball', 'Soccer', 'Outdoor Track',
       'COMPETITION CHEER - Start Date', 'FOOTBALL - Start Date',
       'VOLLEYBALL - Start Date', 'CROSS COUNTRY - Start Date',
       'FIELD HOCKEY - Start Date', 'FCS', 'SATs', 'Esports Fall',
       'Track Camp', 'All County Chorus', 'Parent/Teacher Conferences',
       'VBODA Marching Band Assessment', 'Fall Concert',
       'Middle School Football Game', 'Regional Orchestra',
       'Boys Basketball -', 'Swim -', 'Indoor Track -',
       'Girls Basketball -', 'Wrestling -',
       'VAYFA - Youth Football Games', 'Indoor Percussion',
       'Fall Band Performance', 'Pics', 'TMS Choir Concert',
       'Virginia Jags - Youth Football Games', 'Mus', 'TMS Band Concert',
       'Orche

In [11]:
# List of allowed sport keywords (substrings)
allowed_sports = [
    "Golf", "Field Hockey", "Football", "Volleyball", "Sideline Cheer Fall", 
    "Cross Country", "Competition Cheer", "Basketball", "Wrestling", 
    "Swim & Dive", "Indoor Track", "Lacrosse", "Tennis", "Softball", 
    "Baseball", "Soccer", "Outdoor Track", "Swim", 
    "Girls Soccer", "Girls Basketball", "Girls Lacrosse", 
    "Boys Soccer", "Boys Lacrosse", "Boys Basketball"
]

# Build regex pattern from keywords
pattern = '|'.join(allowed_sports)

# Keep only rows where Sport contains one of the allowed substrings
sport_calendar_df = sport_calendar_df[sport_calendar_df['Sport'].str.contains(pattern, case=False, na=False)]

# Reset index for cleanliness
sport_calendar_df = sport_calendar_df.reset_index(drop=True)

print(sport_calendar_df['Sport'].unique())

['Golf' 'Field Hockey' 'Football' 'Volleyball' 'Sideline Cheer Fall'
 'Cross Country' 'Competition Cheer' 'Basketball' 'Wrestling'
 'Swim & Dive' 'Indoor Track' 'Lacrosse' 'Tennis' 'Softball' 'Baseball'
 'Soccer' 'Outdoor Track' 'COMPETITION CHEER - Start Date'
 'FOOTBALL - Start Date' 'VOLLEYBALL - Start Date'
 'CROSS COUNTRY - Start Date' 'FIELD HOCKEY - Start Date'
 'Middle School Football Game' 'Boys Basketball -' 'Swim -'
 'Indoor Track -' 'Girls Basketball -' 'Wrestling -'
 'VAYFA - Youth Football Games' 'Virginia Jags - Youth Football Games'
 'Volleyball Camp' 'American Youth Football (AYF) - Games' 'Soccer Tots'
 'Cross Country Team Dinner' 'Middle School Football Championship Game'
 'Swim Team - Pasta Dinner'
 'Volleyball - Varsity - End of Season Celebration' 'Soccer Officials'
 'Seniors vs Staff Basketball Game' 'Boys Basketball' 'Boys Soccer'
 'Girls Basketball' 'Boys soccer interest' 'Baseball Fall Ball'
 'Girls Soccer Interest' 'Swim' 'Girls Soccer'
 'Field Hockey Awards 

In [13]:
import re
import numpy as np
import pandas as pd

# --- 1) Average roster sizes (edit if you want different numbers) ---
avg_roster_sizes = {
    "Football Boys": 50,
    "Cheerleading Girls": 20,          # used for Sideline/Competition Cheer too
    "Field Hockey Girls": 20,
    "Soccer Boys": 20, "Soccer Girls": 20,
    "Cross Country Boys": 25, "Cross Country Girls": 25,
    "Swimming Boys": 20, "Swimming Girls": 20,  # swim & dive mapped here
    "Golf Boys": 8, "Golf Girls": 8,
    "Volleyball Boys": 12, "Volleyball Girls": 12,
    "Basketball Boys": 13, "Basketball Girls": 13,
    "Wrestling Boys": 25,
    "Baseball Boys": 16,
    "Softball Girls": 16,
    "Tennis Boys": 10, "Tennis Girls": 10,
    "Track & Field Boys": 40, "Track & Field Girls": 40,
    "Lacrosse Boys": 25, "Lacrosse Girls": 25,
}

# --- 2) Helpers: normalize text & detect gender tokens ---
def _norm(s: str) -> str:
    s = str(s).lower()
    s = s.replace("&", " and ")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _detect_gender(norm_text: str):
    if re.search(r"\bboys?\b|\bboy'?s\b", norm_text):
        return "Boys"
    if re.search(r"\bgirls?\b|\bgirl'?s\b", norm_text):
        return "Girls"
    return None  # unspecified/coed

# --- 3) Patterns to recognize sports (add synonyms as needed) ---
SPORT_PATTERNS = [
    (r"\bfootball\b",                        "Football",      "boys"),
    (r"\bfield\s*hockey\b",                  "Field Hockey",  "girls"),
    (r"\bvolley\s*ball\b|\bvolleyball\b",    "Volleyball",    "detect"),
    (r"\bcheer(leading)?\b|sideline\s*cheer|competition\s*cheer", "Cheerleading", "girls"),
    (r"\bwrestling\b|\bwrestle\b",           "Wrestling",     "boys"),
    (r"\bcross\s*country\b|\bxc\b|\bx\s*country\b", "Cross Country", "detect"),
    (r"\bgolf\b",                             "Golf",          "detect"),
    (r"\bbasket\s*ball\b|\bbasketball\b",     "Basketball",    "detect"),
    (r"\bswim(?:ming)?(?:\s*and\s*dive)?\b|\bswim\s*and\s*dive\b|\bdive\b", "Swimming", "detect"),
    (r"\btennis\b",                           "Tennis",        "detect"),
    (r"\blacrosse\b|\blax\b",                 "Lacrosse",      "detect"),
    (r"\bbase\s*ball\b|\bbaseball\b",         "Baseball",      "boys"),
    (r"\bsoft\s*ball\b|\bsoftball\b",         "Softball",      "girls"),
    (r"\bsoc(?:cer)?\b|\bsoccer\b",           "Soccer",        "detect"),
    (r"\bindoor\s*track\b",                   "Track & Field", "detect"),
    (r"\boutdoor\s*track\b",                  "Track & Field", "detect"),
    (r"\btrack\b|\btrack\s*and\s*field\b|\bt\s*&\s*f\b", "Track & Field", "detect"),
]

# --- 4) Canonicalize one row to "Sport Gender" (or BOTH) ---
def infer_canonical_sport(event_name: str) -> str | None:
    s = _norm(event_name)
    gender = _detect_gender(s)  # "Boys" / "Girls" / None

    for pat, base, grule in SPORT_PATTERNS:
        if re.search(pat, s):
            # sports that are inherently gendered by name:
            if base == "Baseball":
                return "Baseball Boys"
            if base == "Softball":
                return "Softball Girls"
            if base == "Field Hockey":
                return "Field Hockey Girls"

            # volleyball sometimes has boys teams; honor explicit gender if present
            if base == "Volleyball" and gender in ("Boys", "Girls"):
                return f"Volleyball {gender}"

            # cheer variants → Cheerleading Girls
            if base == "Cheerleading":
                return "Cheerleading Girls"

            # sports where gender can be detected or coed:
            if grule == "boys":
                return f"{base} Boys"
            if grule == "girls":
                return f"{base} Girls"
            if grule == "detect" and gender in ("Boys", "Girls"):
                return f"{base} {gender}"

            # If still unspecified/coed, count BOTH genders where applicable:
            if base in {"Soccer","Basketball","Lacrosse","Tennis","Golf",
                        "Cross Country","Swimming","Track & Field","Volleyball"}:
                return f"{base} BOTH"

            # default fallbacks for single-gender sports:
            if base in {"Football","Wrestling"}:
                return f"{base} Boys"

            # otherwise give up
            return None

    return None  # no sport match found

# --- 5) Add canonical sport + participants to your dataframe ---
# Assumes your combined df is named `sport_calendar_df` and has a column with event titles.
# If your sport title column is not exactly 'Sport', change the name below.
title_col = "Sport"

sport_calendar_df["canonical_sport"] = sport_calendar_df[title_col].apply(infer_canonical_sport)

def participants_from_canonical(canon: str | None) -> float:
    if not canon:
        return np.nan
    if canon.endswith(" BOTH"):
        base = canon.replace(" BOTH", "")
        m = avg_roster_sizes.get(f"{base} Boys", 0)
        f = avg_roster_sizes.get(f"{base} Girls", 0)
        val = (m if m else 0) + (f if f else 0)
        return np.nan if val == 0 else val
    return avg_roster_sizes.get(canon, np.nan)

sport_calendar_df["participants"] = sport_calendar_df["canonical_sport"].apply(participants_from_canonical)

# --- 6) (Optional) See what didn’t match so you can refine patterns quickly ---
unmatched = (
    sport_calendar_df[sport_calendar_df["participants"].isna()][title_col]
    .value_counts()
    .head(30)
)
print("Top unmatched event titles (sample):\n", unmatched)

# --- 7) (Optional) Get busiest days of week by weighted participants ---
# Make sure you already created the 'day_of_week' column as earlier.
busiest = (
    sport_calendar_df.dropna(subset=["participants"])
    .groupby("Weekday", sort=False)["participants"]
    .sum()
    .sort_values(ascending=False)
)
print("\nBusiest days (weighted by participants):\n", busiest)


Top unmatched event titles (sample):
 Series([], Name: count, dtype: int64)

Busiest days (weighted by participants):
 Weekday
Tuesday      24995
Wednesday    23659
Thursday     23423
Friday       22389
Monday       19304
Saturday     12239
Sunday         450
Name: participants, dtype: int64


In [15]:
# Count events per weekday
weekday_counts = sport_calendar_df['Weekday'].value_counts().sort_index()

print(weekday_counts)


Weekday
Friday       699
Monday       669
Saturday     303
Sunday        10
Thursday     794
Tuesday      879
Wednesday    666
Name: count, dtype: int64


In [17]:
# Count sports on Wednesday
wed_sports = (
    sport_calendar_df[sport_calendar_df['Weekday'] == "Wednesday"]
    ['Sport'].value_counts()
)

print("Sports on Wednesday:\n", wed_sports)

Sports on Wednesday:
 Sport
Football                    69
Cross Country               64
Tennis                      54
Outdoor Track               52
Baseball                    51
Soccer                      37
Lacrosse                    34
Boys Basketball             33
Wrestling                   32
Golf                        31
Softball                    31
Basketball                  29
Field Hockey                27
Volleyball                  26
Indoor Track                24
Girls Soccer                17
Boys Soccer                 15
Girls Basketball            12
Swim                         9
Competition Cheer            4
Swim & Dive                  3
Sideline Cheer Fall          3
Baseball Fall Ball           3
POMS Football                3
Swim Team - Pasta Dinner     1
Soccer Tots                  1
Girls Soccer Interest        1
Name: count, dtype: int64


In [19]:
# Filter for Lacrosse events
lax_events = sport_calendar_df[sport_calendar_df['Sport'].str.contains("Lacrosse", case=False, na=False)]

# Get counts of which weekdays Lacrosse events fall on
lax_by_day = lax_events['Weekday'].value_counts().reindex(
    ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"],
    fill_value=0
)

print(lax_by_day)

Weekday
Monday       31
Tuesday      19
Wednesday    34
Thursday     28
Friday       11
Saturday      0
Sunday        0
Name: count, dtype: int64


In [21]:
import pandas as pd

def keep_evenings_and_tbd(df):
    # Copy so we don't overwrite original
    df = sport_calendar_df.copy()

    # Normalize column to string
    df['Time'] = df['Time'].astype(str).str.strip()

    # Identify TBD rows
    is_tbd = df['Time'].str.contains("TBD", case=False, na=False)

    # Convert to datetime (coerce errors so "TBD" -> NaT)
    times = pd.to_datetime(df.loc[~is_tbd, 'Time'], format='%I:%M %p', errors='coerce')

    # Keep if time > 4:00 PM or if it's TBD
    mask = is_tbd | (times.dt.hour > 16) | ((times.dt.hour == 16) & (times.dt.minute > 0))

    return df[mask].reset_index(drop=True)

# Apply to your combined dataset
evening_df = keep_evenings_and_tbd(sport_calendar_df)

print(evening_df[['Date', 'Time', 'Sport']].head())


        Date Time       Sport
0 2024-08-05  TBD        Golf
1 2024-08-07  TBD        Golf
2 2024-09-07  TBD  Volleyball
3 2024-09-25  TBD        Golf
4 2024-10-02  TBD        Golf


In [23]:
# Count NaNs per column
missing_counts = evening_df.isna().sum()

print(missing_counts)


Sport              0
Date               0
Time               0
School             0
Weekday            0
canonical_sport    0
participants       0
dtype: int64


In [25]:
# Remove any rows where Time contains "TBD"
sport_calendar_df = sport_calendar_df[~sport_calendar_df['Time'].str.contains("TBD", case=False, na=False)]

# Reset index for cleanliness
sport_calendar_df = sport_calendar_df.reset_index(drop=True)

print(sport_calendar_df.head())

          Sport       Date    Time                 School    Weekday  \
0          Golf 2024-08-14  4:00pm  Riverbend High School  Wednesday   
1  Field Hockey 2024-08-14  5:00pm  Riverbend High School  Wednesday   
2  Field Hockey 2024-08-14  6:30pm  Riverbend High School  Wednesday   
3  Field Hockey 2024-08-14  6:30pm  Riverbend High School  Wednesday   
4          Golf 2024-08-15  4:00pm  Riverbend High School   Thursday   

      canonical_sport  participants  
0           Golf BOTH            16  
1  Field Hockey Girls            20  
2  Field Hockey Girls            20  
3  Field Hockey Girls            20  
4           Golf BOTH            16  


In [27]:
sport_calendar_df["participants"] = sport_calendar_df["canonical_sport"].apply(participants_from_canonical)

# --- 6) (Optional) See what didn’t match so you can refine patterns quickly ---
unmatched = (
    sport_calendar_df[sport_calendar_df["participants"].isna()][title_col]
    .value_counts()
    .head(30)
)
print("Top unmatched event titles (sample):\n", unmatched)

# --- 7) (Optional) Get busiest days of week by weighted participants ---
# Make sure you already created the 'day_of_week' column as earlier.
busiest = (
    sport_calendar_df.dropna(subset=["participants"])
    .groupby("Weekday", sort=False)["participants"]
    .sum()
    .sort_values(ascending=False)
)
print("\nBusiest days (weighted by participants):\n", busiest)


Top unmatched event titles (sample):
 Series([], Name: count, dtype: int64)

Busiest days (weighted by participants):
 Weekday
Tuesday      24530
Thursday     22748
Wednesday    22602
Friday       21442
Monday       18935
Saturday     11291
Sunday         450
Name: participants, dtype: int64
